## **Text2Speech T2S API**

This tutorial uses ondewo-t2s-api to:

*   List the possible pipelines that can be used for synthesizing
*   List the possible languages that can be used in the synthesize process
*   List the possible domains
*   Synthesize a text to audio
*   Synthesize a batch of texts to audios
*   Manipulate pipelines (Create, Delete, Update, Get)


The first step is to install and the Ondewo Text to Speech client and the requirements needed to interact with the server



In [ ]:
import os
! pwd
! ls
! rm -rf ondewo-t2s-client-python
! git clone https://github.com/ondewo/ondewo-t2s-client-python.git
! cd ondewo-t2s-client-python && git checkout tags/3.1.0 -b 3.1.0 && pip install -r requirements.txt  && git status

In [ ]:
os.getcwd()
! cd ondewo-t2s-client-python && python -m pip install .

In [ ]:
import os
import io
import numpy
import soundfile as sf
import IPython.display as ipd
import grpc
from ondewo.t2s import text_to_speech_pb2, text_to_speech_pb2_grpc
from google.protobuf.json_format import ParseDict, MessageToDict, MessageToJson

Afterwards, the client's objects classes need to be imported

In [ ]:
from ondewo.t2s import text_to_speech_pb2
from ondewo.t2s.client.client import Client
from ondewo.t2s.client.client_config import ClientConfig
from ondewo.t2s.client.services.text_to_speech import Text2Speech
from ondewo.t2s.text_to_speech_pb2 import ListT2sPipelinesRequest, Text2SpeechConfig 
from ondewo.t2s.text_to_speech_pb2 import ListT2sLanguagesRequest, ListT2sDomainsRequest
from ondewo.t2s.text_to_speech_pb2 import T2sPipelineId, RequestConfig, SynthesizeRequest
from ondewo.t2s.text_to_speech_pb2 import BatchSynthesizeRequest

## Connect to the Text to Speech Service

The example below shows how to create a secure channel for a text to speech stub object. When setting *use_secure_channel=True*, a grpc certificate *grpc_cert* is required.

In [ ]:
# credentials = grpc.ssl_channel_credentials(root_certificates=cert)

MAX_MESSAGE_LENGTH: int = 60000000
GRPC_HOST: str = ""
GRPC_PORT: str = ""
CHANNEL: str = f"{GRPC_HOST}:{GRPC_PORT}"
cert=""


options = [
    ('grpc.max_send_message_length', MAX_MESSAGE_LENGTH),
    ('grpc.max_receive_message_length', MAX_MESSAGE_LENGTH),
]

# channel = grpc.secure_channel(CHANNEL, credentials, options=options)

config: ClientConfig = ClientConfig(
  host=GRPC_HOST,
  port=GRPC_PORT, 
  grpc_cert=cert)
    
print(config)
    
client: Client = Client(config=config, use_secure_channel=True)


## Get the service information
In order to get the service information, the following method can be executed. This last will retrieve the release version of the service.

In [ ]:
client.services.text_to_speech.get_service_info()

## List all existing text to speech pipelines

All relevant configurations of the text to speech server are defined in a text to speech pipeline. A running server can store several of such configurations at the same time, and the client can chose which one to pick when he/she sends a request to synthesize a text or batch of texts.

The example below shows how to list all available pipelines by calling the *ListT2sPipelines* function, which takes a *ListT2sPipelinesRequest* as an argument and retrieves a *ListT2sPipelinesResponse*.

In [ ]:
pipelines = client.services.text_to_speech.list_t2s_pipelines(request=ListT2sPipelinesRequest())
pipelines

## List all possible synthesizying languages

A running server can list all possible languages fulfilling specified requirements that can be used to synthesize.

The example below shows how to list all available languages by calling the *ListT2sLanguages* function, which takes a *ListT2sLanguagesRequest* as an argument and retrieves a *ListT2sLanguagesResponse*.


In [ ]:
request = ListT2sLanguagesRequest(speaker_sexes=['female'])
response = client.services.text_to_speech.list_t2s_languages(request=request)
response

## List all possible domains

A running server can list all possible domains fulfilling specified requirements that can be used to synthesize.

The example below shows how to list all available domains by calling the *ListT2sDomains* function, which takes a *ListT2sDomainsRequest* as an argument and retrieves a *ListT2sDomainsResponse*.


In [ ]:
request = ListT2sDomainsRequest(languages=['en'])
response = client.services.text_to_speech.list_t2s_domains(request=request)
response

# Make a synthesize request to the server

The running server offers a feature for synthesizying a text into a audio. In order to make use of it, the Synthesize method is utilized. This method will receive a SynthesizeRequest and retrieve a SynthesizeResponse.



The following pipelines were chosen to examplify.
Linda's voice is choosen for the english voice and Alexandra's voice for german. Therefore, both pipelines need to be asked for to the stab. 

In [ ]:
english_pipeline = T2sPipelineId(id='linda')

In [ ]:
german_pipeline = T2sPipelineId(id='alexandra')

The example below shows how to synthesize a text into an audio.
1.   A configuration has to be created with a *RequestConfig*, specifying the desired optional parameters.
2.   A request has to be created with a *SynthesizeRequest*, specifying the text to be synthesize and the previously created configuration.
3.   By calling the Synthesize method with the created request, the text is synthesized with the specfified configuration.



The following example was created with Alexandra's voice, so as to get an english speaker pronunciation in the audio for the text synthesized. 

In [ ]:
config = RequestConfig(t2s_pipeline_id=english_pipeline.id, 
                       length_scale = 1.0)
request = SynthesizeRequest(text="Hi, this is Alexandra. Thanks for calling. I'm not here at the moment, so please leave a message and I'll call you back.", 
                            config=config)
response = client.services.text_to_speech.synthesize(request=request)
bio = io.BytesIO(response.audio)
audio = sf.read(bio, )
ipd.Audio(audio[0], rate=audio[1])

### Length scale configuration
The attribute length_scale can be finetuned in order to speed up or slow down the audio. 
In the next example the length_scale attribute is set to 0.5 so the retrieved audio will be twice as fast as the original.

In [ ]:
config = RequestConfig(t2s_pipeline_id=english_pipeline.id, 
                       length_scale = 0.5)
request = SynthesizeRequest(text="Hi, this is Alexandra. Thanks for calling. I'm not here at the moment, so please leave a message and I'll call you back.",
                            config=config)
response = client.services.text_to_speech.synthesize(request=request)
bio = io.BytesIO(response.audio)
audio = sf.read(bio, )
ipd.Audio(audio[0], rate=audio[1])

Next, the same attribute is being set to 2.0, therefore, the retrieved audio will be half as fast in comparison to the original.

In [ ]:
config = RequestConfig(t2s_pipeline_id=english_pipeline.id, 
                       length_scale = 2.0)
request = SynthesizeRequest(text="Hi, this is Alexandra. Thanks for calling. I'm not here at the moment, so please leave a message and I'll call you back.",
                            config=config)
response = client.services.text_to_speech.synthesize(request=request)

bio = io.BytesIO(response.audio)
audio = sf.read(bio, )
ipd.Audio(audio[0], rate=audio[1])

### Audio Format confguration

Another attribute that can be configurated is the audio_format.
Audio Format can the setted to:

- 0 for wav
- 1 for flac
- 2 for caf (Core audio format)
- 3 for mp3
- 4 for acc (Advanced audio coding)
- 5 for ogg
- 6 for wma (Windows media audio)

In the following example, the attribute audio_format is setted to create a wav audio file.

In [ ]:
config = RequestConfig(t2s_pipeline_id=english_pipeline.id, 
                       audio_format = 0)
request = SynthesizeRequest(text="Hi, this is Alexandra. Thanks for calling. I'm not here at the moment, so please leave a message and I'll call you back.",
                            config=config)
response = client.services.text_to_speech.synthesize(request=request)
bio = io.BytesIO(response.audio)
audio = sf.read(bio, )
ipd.Audio(audio[0], rate=audio[1])

### Pulse Code Modulation

The pcm attribute represents the number of pulses created for the audio file.
A pcm signal is a sequence of digital audio samples containing the data providing the necessary information to reconstruct the original analog signal.

- 0 for 16 (16  bits per sample)
- 1 for 24 (24  bits per sample)
- 2 for 32 (32  bits per sample)
- 3 for S8
- 4 for U8
- 5 for Float
- 6 for Double

The number of bit per sample affects the quality and size of the retrieved file. As the number of bits per sample increase, so does the size and quality of the file.

In the first example the audio file is generated with the best possible quality and in the second, with the lowest.

In [ ]:
config = RequestConfig(t2s_pipeline_id=english_pipeline.id, pcm = 0)
request = SynthesizeRequest(text="Hi, this is Alexandra. Thanks for calling. I'm not here at the moment, so please leave a message and I'll call you back.",
                            config=config)
response = client.services.text_to_speech.synthesize(request=request)
bio = io.BytesIO(response.audio)
audio = sf.read(bio, )
ipd.Audio(audio[0], rate=audio[1])

In [ ]:
config = RequestConfig(t2s_pipeline_id=english_pipeline.id, pcm = 4)
request = SynthesizeRequest(text="Hi, this is Alexandra. Thanks for calling. I'm not here at the moment, so please leave a message and I'll call you back.",
                            config=config)
response = client.services.text_to_speech.synthesize(request=request)
bio = io.BytesIO(response.audio)
audio = sf.read(bio, )
ipd.Audio(audio[0], rate=audio[1])

## Get Pipeline

In order to get an specific pipeline configuration the GetT2sPipeline method is used. This method received a T2sPipelineId and retrieves a Text2SpeechConfig.

In [ ]:
request = T2sPipelineId(id=german_pipeline.id)
pipeline_config = client.services.text_to_speech.get_t2s_pipeline(request=request)

## Create Pipeline

The server provides a method for creating new pipelines. This can be done with the function CreateT2sPipeline, which receives a Text2SpeechConfig and retrieves a T2sPipelineId.

In [ ]:
new_inference_config = pipeline_config.inference
new_inference_config.composite_inference.text2mel.glow_tts.length_scale = 2
request = Text2SpeechConfig(id='alexandra_2.0',                           # Pipeline Id
                            description=pipeline_config.description,      # Pipeline Description 
                            active=True,                                  # Pipeline is active or not
                            inference=new_inference_config,               # Pipeline Inference configuration
                            normalization=pipeline_config.normalization,  # Pipeline Normalization Parameters
                            postprocessing=pipeline_config.postprocessing)# Pipeline Postprocessing
new_pipeline_id = client.services.text_to_speech.create_t2s_pipeline(request=request)

## Update Pipeline

The server provides a method to update a pipeline called UpdateT2sPipeline, receiving a pipeline configuration.

In the following example, the retrieved configuration in the previous call is modified and used to update the pipeline.

In [ ]:
pipeline_config.inference.composite_inference.text2mel.glow_tts.length_scale = 2

In [ ]:
client.services.text_to_speech.update_t2s_pipeline(request=pipeline_config)

## Delete Pipeline

A pipeline can be deleted with the method DeleteT2sPipeline, receiving a pipeline id.

In [ ]:
request = T2sPipelineId(id='alexandra_2.0')
pipeline_config = client.services.text_to_speech.get_t2s_pipeline(request=request)
client.services.text_to_speech.delete_t2s_pipeline(request=request)